In [1]:
import pandas as pd
import numpy as np
import polars as pl


In [2]:
folder = '.\\input_files\\'
file = '952294234_adventist-health---bakersfield_standardcharges.csv'

dtypes = {
    'Procedure Code': str, 
    'Revenue Code': str, 
    'Modifier1': str,
    'Modifier2': str,   
    'Rx Unit Multiplier': str 
}
df = pd.read_csv(folder + file, dtype=dtypes, skiprows=3)


C:\Users\adria\AppData\Local\Temp\ipykernel_2024\346380062.py:11: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder + file, dtype=dtypes, skiprows=3)


In [3]:
df.rename(columns={
    'Procedure Code': 'local_code',
    'Procedure Description': 'description',
    'Price Tier': 'setting',
    'Revenue Code': 'rev_code',
    'CPT HCPCS Code': 'code',
    'NDC Code': 'ndc',
    'Rx Unit Multiplier': 'drug_hcpcs_multiplier'
    
}, inplace=True)

In [4]:
df = df[~df['local_code'].isin(['Diagnosis Related Group Code', 'Shoppable Services Code'])]

In [5]:
cols = df.columns.tolist()
id_vars = cols[:9]
value_vars = cols[9:]


df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='rate')

In [6]:
df['ndc'].fillna(pd.NA, inplace=True)

In [7]:
mapping = {
    'Gross Charge': 'gross',
    'Discounted Cash Price': 'cash',
    'De-identified minimum negotiated charge': 'min',
    'De-identified maximum negotiated charge': 'max'
}

df['payer_category'] = df['payer'].map(mapping).fillna('payer')

In [8]:
df['plan'] = ''
df['plan'] = np.where(df['payer'].str.contains('ALL_PAYER_-_ALL_OTHER_PLANS'), 'ALL PAYER - ALL OTHER PLANS',
                      np.where(df['payer'].str.contains('ALL_OTHER_PLANS'), 'ALL OTHER PLANS',
                               np.where(df['payer'].str.contains('ALL_PLANS'), 'ALL PLANS', df['plan'])))


df['payer'] = df['payer'].str.replace('ALL_PAYER_-_ALL_OTHER_PLANS|ALL_PLANS|ALL_OTHER_PLANS', '', regex=True).str.strip('-_')

In [9]:
df.loc[df['drug_hcpcs_multiplier'] == '0', 'drug_hcpcs_multiplier'] = pd.NA

In [10]:
df['modifiers'] = df['Modifier1'] + df['Modifier2']
df.drop(['Modifier1', 'Modifier2'], axis=1, inplace=True)

In [11]:
cols = ['setting', 'code', 'ndc', 'local_code', 'plan', 'modifiers']
df[cols] = df[cols].astype(str)
df.loc[:, cols].fillna('', inplace=True)

In [12]:
df['setting'].fillna('', inplace=True)

In [13]:
df['rate'] = df['rate'].astype(float)

In [14]:
df.loc[df['code'].str.match(r'^\d{3}$'), 'ms_drg'] = df['code']

In [15]:
df.loc[df['ndc'].isin(['<NA>', 'nan']), 'ndc'] = ''
df.loc[df['setting'].isin(['<NA>', 'nan']),'setting'] = ''
df.loc[df['modifiers'].isin(['<NA>', 'nan']),'modifiers'] = ''

df['ms_drg'].fillna('', inplace=True)
df['rev_code'].fillna('', inplace=True)

df['drug_hcpcs_multiplier'].fillna(pd.NA, inplace=True)

In [19]:
df.loc[df['local_code'] != df['code'], 'temp_cpt'] = df['code']

In [26]:
df.loc[(df['local_code'] != df['code']) & (df['code'].str.len() == 5), 'hcpcs_cpt'] = df['code']
df.loc[(df['local_code'] != df['code']) & (df['code'].str.len() == 3), 'ms_drg'] = df['code']
df.loc[(df['local_code'] == df['code']) & (df['code'].str.len() != 5) & (df['code'].str.len() != 3), 'icd'] = df['code']

In [38]:
setting_map = {
        'ambulatory surgical': 'outpatient',
        'emergency': 'emergency',
        'observation': 'outpatient',
        'urgent care': 'outpatient',
        'profee': 'inpatient',
        'inpatient': 'inpatient',
        'outpatient': 'outpatient'
    }

df.loc[df['setting'].isin(setting_map.keys()), 'additional_payer_notes'] = f"Original patient setting: {df['setting']}"

df['setting'] = df['setting'].map(setting_map).fillna('1')

In [40]:
df.loc[~df['additional_payer_notes'].isna()]

,local_code,description,setting,rev_code,code,ndc,drug_hcpcs_multiplier,payer,rate,payer_category,plan,modifiers,ms_drg,temp_cpt,icd,hcpcs_cpt,additional_payer_notes
